In [2]:

# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time
import shutil
import json
import glob
import pandas as pd
import os

from lib import eval_helper
from lib import utils
from lib import slurm
from lib.slurm import run_sbatch_job


Running on host:  submit-1


In [6]:
params = {
    "eval_plan": "Llama31_336px",
    "json_config": "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json",
    "checkpoint_dir": "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    "benchmark_name": "vqa",
    "checkpoint_id": "2000"
}

job = run_job(
    sbatch_base_script="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    sbatch_overwrite={
        "job-name": "eval_" + params["benchmark_name"]
    },
    positional_env_vars=list(params.values())
)

job

sbatch --parsable --job-name=eval_vqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch vqa 2000


9675

# Eval plan

## Llama31

In [42]:
# Llama31
EVAL_PLAN = "Llama31_336px"
BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"
checkpoint_start = 2000
checkpoint_end = 8000
checkpoint_int = 1000

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in range(checkpoint_start, checkpoint_end + checkpoint_int, checkpoint_int):
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_sbatch_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(f'job_dict_{EVAL_PLAN}.json', 'w') as f:
    json.dump(job_dict, f, indent=4)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 2000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 3000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 4000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json 

In [113]:
# Llama31
EVAL_PLAN = "Llama31_336px_8500_9500"
checkpoints = [8500, 9000, 9500]

job_dict_json = f'job_dict_{EVAL_PLAN}.json'
assert not os.path.exists(job_dict_json)

BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in checkpoints:
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(job_dict_json, 'w') as f:
    json.dump(job_dict, f, indent=4)

print(f"Saved job_dict to {job_dict_json}")

AssertionError: 

In [73]:
job_dict

{'mmmu': {2000: 9759,
  3000: 9760,
  4000: 9761,
  5000: 9762,
  6000: 9763,
  7000: 9764,
  8000: 9765},
 'docvqa': {2000: 9766,
  3000: 9767,
  4000: 9768,
  5000: 9769,
  6000: 9770,
  7000: 9771,
  8000: 9772},
 'mathvista': {2000: 9773,
  3000: 9774,
  4000: 9775,
  5000: 9776,
  6000: 9777,
  7000: 9778,
  8000: 9779},
 'ai2d': {2000: 9780,
  3000: 9781,
  4000: 9782,
  5000: 9783,
  6000: 9784,
  7000: 9785,
  8000: 9786},
 'chartqa': {2000: 9787,
  3000: 9788,
  4000: 9789,
  5000: 9790,
  6000: 9791,
  7000: 9792,
  8000: 9793},
 'vqa': {2000: 9794,
  3000: 9795,
  4000: 9796,
  5000: 9797,
  6000: 9798,
  7000: 9799,
  8000: 9800},
 'textvqa': {2000: 9801,
  3000: 9802,
  4000: 9803,
  5000: 9804,
  6000: 9805,
  7000: 9806,
  8000: 9807},
 'infographics_w_ocr': {2000: 9808,
  3000: 9809,
  4000: 9810,
  5000: 9811,
  6000: 9812,
  7000: 9813,
  8000: 9814},
 'infographics': {2000: 9815,
  3000: 9816,
  4000: 9817,
  5000: 9818,
  6000: 9819,
  7000: 9820,
  8000: 9821},
 'm

## Llama31 - 10000

In [3]:
job_dict_lm31_10k = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="Llama31_10k",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    checkpoints=[10000],
    # benchmarks=["mathvista"]
)

job_dict_lm31_10k

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mathvista 10000
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31

{'mmmu': {10000: 10008},
 'docvqa': {10000: 10009},
 'mathvista': {10000: 10010},
 'ai2d': {10000: 10011},
 'chartqa': {10000: 10012},
 'vqa': {10000: 10013},
 'textvqa': {10000: 10014},
 'infographics_w_ocr': {10000: 10015},
 'infographics': {10000: 10016},
 'mmbench': {10000: 10017}}

## Llama31 - fb

In [6]:
job_dict_fb_lm31 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="FBL_Llama31",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/fb_llama3.1",
    checkpoints=[8000, 10000],
    # benchmarks=["mathvista"]
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 8000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 8000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experime

## MH19

In [1]:
checkpoints = [10500 + i*1000 for i in range(9)]
checkpoints

[10500, 11500, 12500, 13500, 14500, 15500, 16500, 17500, 18500]

In [3]:
job_dict_mh19 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="MH19_10500_18500",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/MH19_336px_128nodes_bz32_resume",
    checkpoints=[10500 + i*1000 for i in range(9)],
    # benchmarks=["mathvista"]
)

RuntimeError: Found existing job_dict: eval/logs/job_dict_MH19_10500_18500.json

In [12]:
df = eval_helper.get_eval_scores(job_dict_mh19)

Got result for mmmu - 10500
Got result for mmmu - 11500
Got result for mmmu - 12500
Got result for mmmu - 13500
Got result for mmmu - 14500
Got result for mmmu - 15500
Got result for mmmu - 16500
Got result for mmmu - 17500
Got result for mmmu - 18500
Got result for docvqa - 10500
Got result for docvqa - 11500
Got result for docvqa - 12500
Got result for docvqa - 13500
Got result for docvqa - 14500
Got result for docvqa - 15500
Got result for docvqa - 16500
Got result for docvqa - 17500
Got result for docvqa - 18500
Got result for mathvista - 10500
Got result for mathvista - 11500
Got result for mathvista - 12500
Got result for mathvista - 13500
Got result for mathvista - 14500
Got result for mathvista - 15500
Got result for mathvista - 16500
Got result for mathvista - 17500
Got result for mathvista - 18500
Got result for ai2d - 10500
Got result for ai2d - 11500
Got result for ai2d - 12500
Got result for ai2d - 13500
Got result for ai2d - 14500
Got result for ai2d - 15500
Got result fo

In [13]:
df.loc[17500]

mmmu/accuracy                                    0.5878
mmmu/mllm_eval_accuracy                          0.5889
docvqa/anls_total_score                          0.6201
docvqa/mllm_evaluation_anls_score                0.6216
mathvista/accuracy                                0.377
ai2d/accuracy                                    0.7704
chartqa/accuracy                                 0.4914
vqa/accuracy                                     0.7035
vqa/mllm_evaluation_accuracy                     0.7256
textvqa/accuracy                                 66.898
textvqa/mllm_eval_accuracy                       71.578
infographics_w_ocr/anls_total_score              0.6256
infographics_w_ocr/mllm_evaluation_anls_score    0.5712
infographics/anls_total_score                    0.4798
infographics/mllm_evaluation_anls_score          0.4177
mmbench/overall                                  0.7422
Name: 17500, dtype: object

# Parse results

In [1]:
# job_dict_file = 'job_dict_Llama31_336px.json'
job_dict_file = 'job_dict_Llama31_336px_8500_9500.json'

with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df = eval_helper.get_eval_scores(job_dict)
df

NameError: name 'json' is not defined

In [4]:
job_dict_file = 'job_dict_FBL_Llama31.json'
with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df2 = eval_helper.get_eval_scores(job_dict)
df2

Got result for mmmu - 8000
Got result for mmmu - 10000
Got result for docvqa - 8000
Got result for docvqa - 10000
Got result for mathvista - 8000
Got result for mathvista - 10000
Got result for ai2d - 8000
Got result for ai2d - 10000
Got result for chartqa - 8000
Got result for chartqa - 10000
Got result for vqa - 8000
Got result for vqa - 10000
Got result for textvqa - 8000
Got result for textvqa - 10000
Got result for infographics_w_ocr - 8000
Got result for infographics_w_ocr - 10000
Got result for infographics - 8000
Got result for infographics - 10000
Got result for mmbench - 8000
Got result for mmbench - 10000


,mmmu/accuracy,mmmu/mllm_eval_accuracy,docvqa/anls_total_score,docvqa/mllm_evaluation_anls_score,mathvista/accuracy,ai2d/accuracy,chartqa/accuracy,vqa/accuracy,vqa/mllm_evaluation_accuracy,textvqa/accuracy,textvqa/mllm_eval_accuracy,infographics_w_ocr/anls_total_score,infographics_w_ocr/mllm_evaluation_anls_score,infographics/anls_total_score,infographics/mllm_evaluation_anls_score,mmbench/overall
8000,0.5067,0.5156,0.5386,0.5411,0.344,0.7121,0.4746,0.6831,0.7137,59.028,65.38,0.5902,0.536,0.4384,0.3746,0.6635
10000,0.4744,0.4822,0.54,0.5432,0.348,0.6953,0.4578,0.6931,0.7224,59.414,65.708,0.592,0.5354,0.4472,0.3785,0.6772


# Stage 2

In [4]:
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
ALIGNER_CODE_DIR="/fsx_0/user/tranx/eval_adel"
# EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval"
EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/eval_adel/llm_mm_aligner/experiments/aws_adel/eval"
# STAGE2_OUTPUT_DIR="/fsx_0/checkpoints/tranx/MM9-Stage2-70B"

## Exp 29

In [20]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: []


In [21]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp")

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800]
Got result for mmmu - 200: [('mmmu/accounting', 0.4), ('mmmu/agriculture', 0.6333333333333333), ('mmmu/architecture_and_engineering', 0.23333333333333334), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.9333333333333333), ('mmmu/basic_medical_science', 0.7), ('mmmu/biology', 0.43333333333333335), ('mmmu/chemistry', 0.4), ('mmmu/clinical_medicine', 0.6), ('mmmu/computer_science', 0.6), ('mmmu/design', 0.7666666666666667), ('mmmu/diagnostics_and_laboratory_medicine', 0.36666666666666664), ('mmmu/economics', 0.6666666666666666), ('mmmu/electronics', 0.26666666666666666), ('mmmu/energy_and_power', 0.5333333333333333), ('mmmu/finance', 0.5333333333333333), ('mmmu/geography', 0.5333333333333333), ('mmmu/history', 0.7333333333333333), ('mmmu/literature', 0.8), ('mmmu/manage', 0.5666666666666667), ('mmmu/marketing', 0.43333333333333335), ('mmmu/m

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.5556,0.5444,0.649,0.391,0.7937,0.4937,0.7255,0.68126,0.641,0.5379,0.7381
400,0.57,0.5656,0.6534,0.399,0.7895,0.4979,0.7258,0.68664,0.6334,0.5391,0.7353
600,0.5567,0.5433,0.6576,0.379,0.7947,0.4977,0.7262,0.68338,0.6451,0.5388,0.737
800,0.5756,0.5622,0.6862,0.435,0.8258,0.5386,0.7527,0.7053,0.6473,0.5585,0.7507
1000,0.5667,0.5511,0.6915,0.441,0.8229,0.5447,0.7517,0.70046,0.6548,0.5557,0.7557
1200,0.5833,0.5778,0.6882,0.443,0.8251,0.5338,0.753,0.69956,0.6509,0.5539,0.7495
1400,0.5633,0.5578,0.6576,0.39,0.8015,0.5053,0.7249,0.68468,0.637,0.5447,0.7389
1600,0.5811,0.5678,0.6634,0.387,0.8018,0.5018,0.7266,0.68846,0.6457,0.5496,0.7391
1800,0.5822,0.57,0.6844,0.479,0.8174,NaN,0.7503,0.70316,0.6526,0.5586,0.7516
2000,0.5522,0.5433,0.6628,0.378,0.8021,0.5066,0.7267,0.68474,0.642,0.5559,0.736


## Exp 28

In [46]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: [4400]
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/checkpoint-4400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/evals/eval_jobs_checkpoint-4400.json


In [45]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28")

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400]
Got result for mmmu - 200: [('mmmu/accounting', 0.5), ('mmmu/agriculture', 0.5666666666666667), ('mmmu/architecture_and_engineering', 0.43333333333333335), ('mmmu/art', 0.7), ('mmmu/art_theory', 0.8666666666666667), ('mmmu/basic_medical_science', 0.6666666666666666), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.4), ('mmmu/clinical_medicine', 0.6333333333333333), ('mmmu/computer_science', 0.6), ('mmmu/design', 0.7), ('mmmu/diagnostics_and_laboratory_medicine', 0.4666666666666667), ('mmmu/economics', 0.5333333333333333), ('mmmu/electronics', 0.43333333333333335), ('mmmu/energy_and_power', 0.6), ('mmmu/finance', 0.4666666666666667), ('mmmu/geography', 0.5333333333333333), ('mmmu/history', 0.7333333333333333), ('mmmu/literature', 0.8333333333333334), ('mmmu/manage', 0.4666666666666667), ('mmmu/marketing', 0.4666666666666667), ('mmmu/materials', 0.53

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.5689,0.5578,0.6852,0.411,0.8174,0.5189,0.7468,0.69688,0.6447,0.5587,0.743
400,0.5767,0.5544,0.6946,0.42,0.818,0.5345,0.7513,0.7067,0.6489,0.5459,0.7529
600,0.58,0.5689,0.6835,0.432,0.8235,0.5301,0.7491,0.70012,0.6515,0.5574,0.743
800,0.5756,0.5622,0.6862,0.435,0.8258,0.5386,0.7527,0.7053,0.6473,0.5585,0.7507
1000,0.5667,0.5511,0.6915,0.441,0.8229,0.5447,0.7517,0.70046,0.6548,0.5557,0.7557
1200,0.5833,0.5778,0.6882,0.443,0.8251,0.5338,0.753,0.69956,0.6509,0.5539,0.7495
1400,0.5967,0.5944,0.693,0.449,0.819,0.536,0.7514,0.7001,0.6499,0.5689,0.7511
1600,0.59,0.5833,0.6895,0.462,0.8167,0.5229,0.7529,0.69826,0.6523,0.5537,0.7524
1800,0.5822,0.57,0.6844,0.479,0.8174,NaN,0.7503,0.70316,0.6526,0.5586,0.7516
2000,0.5811,0.5656,0.69,0.465,0.819,0.5428,0.7523,0.70334,0.6562,0.5637,0.7527


## 504px exp 29

In [47]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: []


In [48]:
# output_dir = "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29"
# c = 5200
# eval_helper.run_eval_plan(
#     eval_base_sbatch=EVAL_SBATCH,
#     aligner_parent_dir=ALIGNER_CODE_DIR,
#     eval_config_dir=EVAL_CONFIG_DIR,
#     checkpoint_dir=output_dir,
#     checkpoints=[c],
#     save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
#     benchmarks=["mmmu"],
#     rerun_if_exists=True,
#     print_cmd=True
# )

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3554613515.py, line 10)

In [49]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29")

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800, 5000, 5200, 5400, 5600]
Got result for mmmu - 200: [('mmmu/accounting', 0.5666666666666667), ('mmmu/agriculture', 0.6), ('mmmu/architecture_and_engineering', 0.23333333333333334), ('mmmu/art', 0.7), ('mmmu/art_theory', 0.8333333333333334), ('mmmu/basic_medical_science', 0.6333333333333333), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.36666666666666664), ('mmmu/clinical_medicine', 0.5666666666666667), ('mmmu/computer_science', 0.5666666666666667), ('mmmu/design', 0.7666666666666667), ('mmmu/diagnostics_and_laboratory_medicine', 0.3333333333333333), ('mmmu/economics', 0.6666666666666666), ('mmmu/electronics', 0.3333333333333333), ('mmmu/energy_and_power', 0.6), ('mmmu/finance', 0.36666666666666664), ('mmmu/geography', 0.6), ('mmmu/history', 0.8), ('mmmu/literature', 0.9), ('mmmu/manage', 0.5333333333333333), ('mmmu/marketing', 0.53333

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.57,0.5611,0.6496,0.4,0.7905,0.5061,0.7205,0.67858,0.6418,0.5387,0.7386
400,0.5767,0.5622,0.653,0.399,0.7976,0.5042,0.7206,NaN,0.6358,0.5427,0.7429
600,0.5611,0.5656,0.6536,0.394,0.7976,0.5135,0.7229,0.68194,0.6392,0.5387,0.7443
800,0.5678,0.5467,0.6537,0.385,0.784,0.5163,0.7204,0.68472,0.644,0.537,0.7468
1000,0.5622,0.55,0.6558,0.385,0.7863,0.5142,0.7228,0.68406,0.6395,0.541,0.7473
1200,0.5611,0.5667,0.6667,0.393,0.7986,0.4978,0.7268,0.68288,0.6425,0.5537,0.7485
1400,0.5456,0.5456,0.6563,0.378,0.7889,0.5108,0.7243,0.6856,0.6416,0.5457,0.7402
1600,0.5567,0.5433,0.6644,0.391,0.7889,0.516,0.7263,0.6871,0.6455,0.5486,0.7419
1800,0.57,0.5478,0.6597,0.399,0.7824,0.5116,0.7249,0.68338,0.6448,0.5458,0.7436
2000,0.5789,0.5644,0.6627,0.385,0.7876,0.5063,0.725,0.68392,0.6347,0.5422,0.7526


## 336px i18n

In [24]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: []


In [19]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n")

[5000, 5200, 5400, 5600, 5800, 6000, 6200, 6400]
Got result for mmmu - 5000: [('mmmu/accounting', 0.6), ('mmmu/agriculture', 0.5666666666666667), ('mmmu/architecture_and_engineering', 0.3), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.8666666666666667), ('mmmu/basic_medical_science', 0.6333333333333333), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.43333333333333335), ('mmmu/clinical_medicine', 0.6333333333333333), ('mmmu/computer_science', 0.4666666666666667), ('mmmu/design', 0.6), ('mmmu/diagnostics_and_laboratory_medicine', 0.4), ('mmmu/economics', 0.7), ('mmmu/electronics', 0.4), ('mmmu/energy_and_power', 0.6333333333333333), ('mmmu/finance', 0.4), ('mmmu/geography', 0.6), ('mmmu/history', 0.7666666666666667), ('mmmu/literature', 0.8666666666666667), ('mmmu/manage', 0.5333333333333333), ('mmmu/marketing', 0.43333333333333335), ('mmmu/materials', 0.3), ('mmmu/math', 0.5), ('mmmu/mechanical_engineering', 0.4), ('mmmu/music', 0.3), ('mmmu/pharmacy', 0.7), ('mm

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
5000,0.5444,0.5433,0.6706,0.476,0.8488,0.7014,0.7256,0.66916,0.6587,0.5542,0.7887
5200,0.5422,0.5278,0.6631,0.524,0.8724,0.7788,0.728,0.66484,0.6527,0.5203,0.7947
5400,0.5444,0.5356,0.6418,0.521,0.8818,0.7923,0.7172,0.65898,0.6535,0.5298,0.7902
5600,0.5278,0.5211,0.6488,NaN,0.8951,0.7982,0.7123,0.65266,NaN,0.54,0.7879
5800,0.55,0.5389,0.6482,0.565,0.8909,0.801,0.7169,0.6532,0.6401,0.5225,0.7844
6000,0.5544,0.5522,0.6226,0.55,0.9028,0.8121,0.7098,0.65218,0.6485,0.5236,0.7864
6200,0.5478,0.5433,0.6321,0.564,0.8957,0.8031,0.7086,0.65124,0.644,0.5278,0.7886
6400,0.5489,0.5444,0.6457,0.581,0.9035,0.8084,0.7016,0.64498,0.6481,0.5346,0.7977
